In [1]:
import glob
import os
import random
import pandas as pd
import shutil
import numpy as np

In [2]:
annotated_files = [f for f in glob.glob("not_totally_annotated\\labels\\*.txt") if os.path.getsize(f) > 0]

In [3]:
print(len(annotated_files))
annotated_files[10]

93


'not_totally_annotated\\labels\\spots_Anglet_GrandePlage_2022-03-28_08-45.txt'

In [4]:
import random
random.shuffle(annotated_files)

In [5]:
import re

def save_img_labels_to_annotated(files, folder):   
    with open('annotated\\all.txt', 'w') as f:
        for file in files:
            new_path = file.replace('not_totally_annotated\\', 'annotated\\')
            new_path = new_path.replace('labels', f'labels\\{folder}')
            if not os.path.exists(new_path):
                labels = open(file, "r").read()
                labels = np.array([[float(x) for x in row.split(" ")] for row in labels.split("\n")])
                labels[:,2] = 1 - labels[:,2]
                labels = labels.tolist()
                with open(new_path, 'w') as corrected_labels:
                    for i in range(len(labels)):
                        for j in range(4):
                            corrected_labels.write("%s " % labels[i][j])
                        corrected_labels.write("%s" % labels[i][4])
                        if i < len(labels) - 1:
                            corrected_labels.write("\n")    

                new_path_image = new_path.replace("labels", "images").replace(".txt", ".jpg")
                image = re.sub(".*?\\\\", "", file.replace(".txt",".jpg"))
                shutil.copy("..\\..\\..\\data\\surf_only_images\\" + image, new_path_image)
                f.write("%s\n" % image)      

In [6]:
n = len(annotated_files)
i_max_train = int(n*0.6)
i_max_val = i_max_train + int(n*0.2)

In [7]:
save_img_labels_to_annotated(annotated_files[:i_max_train], 'train')
save_img_labels_to_annotated(annotated_files[i_max_train:i_max_val], 'val')
save_img_labels_to_annotated(annotated_files[i_max_val:], 'test')